In [1]:
import os
import pickle
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm import tqdm
from typing import collections
warnings.filterwarnings(action='ignore')

In [2]:
# 원본 데이터 read
customer_info = pd.read_csv('../data/LPOINT_BIG_COMP_01_DEMO.csv') #고객정보
purchase_info = pd.read_csv('../data/LPOINT_BIG_COMP_02_PDDE.csv') # 구매정보
product_info = pd.read_csv('../data/LPOINT_BIG_COMP_04_PD_CLAC.csv') #제품 정보

In [3]:
 # 구매정보, 상품정보, 고객정보 join
purchase_joined = pd.merge(purchase_info,product_info)
purchase_joined = pd.merge(purchase_joined,customer_info)

In [5]:
# 식품 제외 
food_category = ['과자', '조리식품', '조미료', '수산물', '음료', '채소', '주류', '축산물', '유제품',
                                '냉장식품', '건해산물', '대용식', '과일', '냉동식품','커피/차', '병통조림', '양곡', '담배', '기타(비상품)']

food_purchase_joined = purchase_joined[purchase_joined["clac_hlv_nm"].isin(food_category)]
purchase_joined = purchase_joined[~purchase_joined["clac_hlv_nm"].isin(food_category)]

# 오프라인 전처리

## 오프라인구매내역 추출

In [6]:
#오프라인 구매내역 추출
purchase_offline_nofood = purchase_joined[purchase_joined['chnl_dv'] == 1]
print(purchase_offline_nofood.shape)

(1047725, 16)


In [7]:
purchase_offline_nofood.head()

,cust,rct_no,chnl_dv,cop_c,br_c,pd_c,de_dt,de_hr,buy_am,buy_ct,pd_nm,clac_hlv_nm,clac_mcls_nm,ma_fem_dv,ages,zon_hlv
0,M430112881,A01000001113,1,A01,A010039,PD0290,20210101,10,15000.0,1,남성티셔츠,남성의류,남성의류상의,여성,50대,Z17
1,M430112881,A01000003148,1,A01,A010039,PD0290,20210101,10,19000.0,1,남성티셔츠,남성의류,남성의류상의,여성,50대,Z17
2,M430112881,A01000003148,1,A01,A010039,PD0290,20210101,10,19000.0,1,남성티셔츠,남성의류,남성의류상의,여성,50대,Z17
3,M430112881,A01000004946,1,A01,A010039,PD0290,20210101,10,19000.0,1,남성티셔츠,남성의류,남성의류상의,여성,50대,Z17
4,M430112881,A01000005297,1,A01,A010039,PD1692,20210101,10,9900.0,1,커피/음료,테넌트/음식점,식당,여성,50대,Z17


In [8]:
purchase_offline_nofood.to_csv('purchase_offline_nofood.csv',index=False)

## 카테고리 1개 구매 고객 csv만들기

In [9]:
try:
    with open("./off_one_custs.pickle", "rb") as f:
        one_custs = pickle.load(f)
except:
    
    off_cust_id = purchase_offline_nofood['cust'].unique()
    one_custs = []
    more_than_one_custs = []
    temp = collections.defaultdict(list)

    for person in tqdm(off_cust_id):
        product_count = purchase_offline_nofood[purchase_offline_nofood["cust"] == person]["clac_hlv_nm"].nunique()
        if product_count <= 1:
            one_custs.append(person)
        else:
            more_than_one_custs.append(person)
            
    print(len(set(one_custs)))
    print(len(set(more_than_one_custs)))
    
    with open("./off_one_custs.pickle", "wb") as f:
        pickle.dump(one_custs, f)

100%|██████████| 25003/25003 [14:43<00:00, 28.29it/s]

2154
22849


## 카테고리 2개 이상 고객 추출

In [9]:
with open("./off_one_custs.pickle", "rb") as f:
    one_custs = pickle.load(f)

In [10]:
# 카테고리 2개 이상 구매한 고객들의 구매 내역 데이터프레임
custs_over_two_df = purchase_offline_nofood[~purchase_offline_nofood['cust'].isin(one_custs)]
print(custs_over_two_df.shape)

(1042905, 16)


## 오프라인 고객 카테고리별 구매금액 합 & 구매개수 합 csv만들기

In [11]:
product_ct_dict = collections.defaultdict(dict)
product_am_dict = collections.defaultdict(dict)

for person in tqdm(custs_over_two_df["cust"].unique()):
    product_ct_dict[person] = {}
    product_am_dict[person] = {}
    
    temp = custs_over_two_df[custs_over_two_df["cust"] == person].groupby(["clac_hlv_nm"])[["buy_am", "buy_ct"]].sum().reset_index()
    
    for value in temp.iterrows():
        product_ct_dict[person][value[1]["clac_hlv_nm"]] = value[1]["buy_ct"] # 오프라인 고객 카테고리별 구매횟수 합 
        product_am_dict[person][value[1]["clac_hlv_nm"]] = value[1]["buy_am"] # 오프라인 고객 카테고리별 구매금액 합 



100%|██████████| 22849/22849 [16:01<00:00, 23.76it/s]


In [21]:
offline_buycnt_final = pd.DataFrame(product_ct_dict).transpose().replace(np.nan, 0).reset_index().rename(columns={"index" : "cust_id"})
offline_buyam_final = pd.DataFrame(product_am_dict).transpose().replace(np.nan, 0).reset_index().rename(columns={"index" : "cust_id"})

In [22]:
offline_buycnt_final.to_csv('./offline_buycnt.csv',index=False)
offline_buyam_final.to_csv('./offline_buyam.csv',index=False)

## 오프라인 고객 카테고리별 구매금액 합 데이터 전처리

In [23]:
# 온라인 고객 1년 구매금액 합(카테고리 2개 이상)
off_buyam = pd.read_csv('./offline_buyam.csv')

### normal 고객 카테고리별 구매금액 합 데이터프레임

In [24]:
# 20만원 이상, 200만원 이하 구매고객 
off_buy_am_sum = purchase_offline_nofood[['cust','buy_am']].groupby('cust').sum()
off_normal = off_buy_am_sum[off_buy_am_sum['buy_am'] < 100000000] #금액으로만 구분
off_normal_ids = off_normal.index.tolist()

#20만원 이상, 200만원 이하, 카테고리2개 이상 구매고객 구매금액 합 데이터
off_buyam_normal = off_buyam[off_buyam['cust_id'].isin(off_normal_ids)].reset_index(drop=True)

### 고객 인구통계학적 정보 합치기

In [25]:
 # 20만원 이상, 200만원 이하, 카테고리 2개 이상 구매 고객 id 데이터 프레임
off_final_normal_ids = off_buyam_normal[['cust_id']].rename(columns={'cust_id':'cust'})

In [26]:
off_normal_demo =  pd.merge(off_final_normal_ids, customer_info).iloc[:,:-1]

In [27]:
gender = pd.get_dummies(off_normal_demo['ma_fem_dv'], drop_first=True)
ages = pd.get_dummies(off_normal_demo['ages'])
off_normal_demo_oh= pd.concat([gender,ages],axis=1)

In [28]:
off_buyam_normal_d = pd.concat([off_buyam_normal,off_normal_demo_oh],axis=1)

### 구매금액 합 클러스터링

In [29]:
from sklearn.preprocessing import Normalizer
from sklearn.cluster import KMeans

In [31]:
# normalizer를 이용해서 스케일링하기
off_buyam_normal_pd = off_buyam_normal_d.drop(['cust_id'], axis=1).copy() # scaling을 위해 cust_id 드롭
col_nm = off_buyam_normal_pd.columns

nom = Normalizer(copy=False)
off_buyam_normal_pd_n = pd.DataFrame(nom.fit_transform(off_buyam_normal_pd))  # normalizer로 스케일링
off_buyam_normal_pd_n.columns = col_nm

In [34]:
kmeans = KMeans(n_clusters = 5, random_state=0)
am_clusters = kmeans.fit(off_buyam_normal_pd_n)

In [35]:
# 구매금액 합 최종
off_buyam_normal_final = off_buyam_normal_d.copy()
off_buyam_normal_final['cluster'] = am_clusters.labels_

# 구매금액(normalized) 합 최종
cid = off_buyam_normal_final[['cust_id']]
cluster = off_buyam_normal_final[['cluster']]
off_buyam_normal_n_final= pd.concat([cid,off_buyam_normal_pd_n],axis=1)
off_buyam_normal_n_final = pd.concat([off_buyam_normal_n_final,cluster],axis=1)

In [36]:
off_buyam_normal_final

,cust_id,구기/필드스포츠,남성의류,문구/사무용품,생활/렌탈서비스,세제/위생,속옷/양말/홈웨어,스포츠패션,아웃도어/레저,주방잡화,...,금융/보험서비스,기타상품,여성,20대,30대,40대,50대,60대,70대,cluster
0,M430112881,49000.0,255800.0,7000.0,100.0,13900.0,10000.0,78000.0,1390.0,26000.0,...,0.0,0.0,1,0,0,0,1,0,0,1
1,M506355556,0.0,727740.0,0.0,0.0,0.0,0.0,247300.0,278000.0,0.0,...,0.0,0.0,1,0,0,1,0,0,0,1
2,M900002349,0.0,2110800.0,6500.0,0.0,0.0,6000.0,165000.0,1500.0,0.0,...,0.0,0.0,1,0,1,0,0,0,0,1
3,M694209751,2900.0,157750.0,17100.0,500.0,88200.0,0.0,1247280.0,179100.0,92570.0,...,0.0,0.0,0,0,0,1,0,0,0,0
4,M123475113,0.0,315940.0,0.0,0.0,0.0,0.0,50000.0,0.0,0.0,...,0.0,0.0,0,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22785,M590235414,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3000.0,...,0.0,0.0,1,0,1,0,0,0,0,0
22786,M569436409,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1,0,0,0,0,1,0,0
22787,M894085775,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0,0,0,0,1,0,0,0
22788,M803205692,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0,1,0,0,0,0,0,0


In [37]:
off_buyam_normal_n_final

,cust_id,구기/필드스포츠,남성의류,문구/사무용품,생활/렌탈서비스,세제/위생,속옷/양말/홈웨어,스포츠패션,아웃도어/레저,주방잡화,...,금융/보험서비스,기타상품,여성,20대,30대,40대,50대,60대,70대,cluster
0,M430112881,0.128711,0.671925,0.018387,0.000263,0.036512,0.026268,0.204887,0.003651,0.068296,...,0.0,0.0,2.626761e-06,0.000000,0.000000e+00,0.000000e+00,0.000003,0.000000,0.0,1
1,M506355556,0.000000,0.829354,0.000000,0.000000,0.000000,0.000000,0.281830,0.316817,0.000000,...,0.0,0.0,1.139629e-06,0.000000,0.000000e+00,1.139629e-06,0.000000,0.000000,0.0,1
2,M900002349,0.000000,0.992731,0.003057,0.000000,0.000000,0.002822,0.077601,0.000705,0.000000,...,0.0,0.0,4.703104e-07,0.000000,4.703104e-07,0.000000e+00,0.000000,0.000000,0.0,1
3,M694209751,0.000598,0.032526,0.003526,0.000103,0.018186,0.000000,0.257171,0.036928,0.019087,...,0.0,0.0,0.000000e+00,0.000000,0.000000e+00,2.061853e-07,0.000000,0.000000,0.0,0
4,M123475113,0.000000,0.986520,0.000000,0.000000,0.000000,0.000000,0.156125,0.000000,0.000000,...,0.0,0.0,0.000000e+00,0.000000,0.000000e+00,3.122492e-06,0.000000,0.000000,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22785,M590235414,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.074976,...,0.0,0.0,2.499211e-05,0.000000,2.499211e-05,0.000000e+00,0.000000,0.000000,0.0,0
22786,M569436409,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,3.855232e-06,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000004,0.0,0
22787,M894085775,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000036,0.000000,0.0,0
22788,M803205692,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000e+00,0.000012,0.000000e+00,0.000000e+00,0.000000,0.000000,0.0,0


In [38]:
off_buyam_normal_final.to_csv('./off_buyam_normal_final.csv',index=False)

In [39]:
# on_buyam_normal_n_final.to_csv('on_buyam_normal_n_final.csv',index=False)

## 오프라인 고객 카테고리별 구매개수 합 데이터 전처리

In [40]:
# 온라인 고객 1년 구매금액 합(카테고리 2개 이상)
off_buycnt = pd.read_csv('./offline_buycnt.csv')

### normal 고객 카테고리별 구매금액 합 데이터프레임

In [41]:
#1000먼원 미만, 카테고리2개 이상 구매고객 구매개수 합 데이터
off_buycnt_normal = off_buycnt[off_buycnt['cust_id'].isin(off_normal_ids)].reset_index(drop=True)

### 고객 인구통계학적 정보 합치기

In [42]:
off_buycnt_normal_d = pd.concat([off_buycnt_normal, off_normal_demo_oh],axis=1)

### 구매개수 클러스터링

In [43]:
# 스케일링
off_buycnt_normal_pd = off_buycnt_normal_d.drop(['cust_id'], axis=1).copy() # scaling을 위해 cust_id 드롭
col_nm = off_buycnt_normal_pd.columns

nom = Normalizer(copy=False)
off_buycnt_normal_pd_n = pd.DataFrame(nom.fit_transform(off_buycnt_normal_pd))  # normalizer로 스케일링
off_buycnt_normal_pd_n.columns = col_nm

In [46]:
kmeans = KMeans(n_clusters = 20, random_state=0)
cnt_clusters = kmeans.fit(off_buycnt_normal_pd_n)

In [47]:
# 구매금액 합 최종
off_buycnt_normal_final = off_buycnt_normal_d.copy()
off_buycnt_normal_final['cluster'] = cnt_clusters.labels_

# 구매금액(normalized) 합 최종
cid = off_buycnt_normal_final[['cust_id']]
cluster = off_buycnt_normal_final[['cluster']]
off_buycnt_normal_n_final= pd.concat([cid, off_buycnt_normal_pd_n],axis=1)
off_buycnt_normal_n_final = pd.concat([off_buycnt_normal_n_final,cluster],axis=1)

In [48]:
off_buycnt_normal_final

,cust_id,구기/필드스포츠,남성의류,문구/사무용품,생활/렌탈서비스,세제/위생,속옷/양말/홈웨어,스포츠패션,아웃도어/레저,주방잡화,...,금융/보험서비스,기타상품,여성,20대,30대,40대,50대,60대,70대,cluster
0,M430112881,1.0,7.0,5.0,1.0,1.0,1.0,2.0,1.0,27.0,...,0.0,0.0,1,0,0,0,1,0,0,14
1,M506355556,0.0,7.0,0.0,0.0,0.0,0.0,3.0,2.0,0.0,...,0.0,0.0,1,0,0,1,0,0,0,11
2,M900002349,0.0,21.0,6.0,0.0,0.0,2.0,2.0,1.0,0.0,...,0.0,0.0,1,0,1,0,0,0,0,11
3,M694209751,1.0,12.0,6.0,1.0,9.0,0.0,16.0,1.0,14.0,...,0.0,0.0,0,0,0,1,0,0,0,12
4,M123475113,0.0,4.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0,0,0,1,0,0,0,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22785,M590235414,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1,0,1,0,0,0,0,2
22786,M569436409,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1,0,0,0,0,1,0,16
22787,M894085775,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0,0,0,0,1,0,0,16
22788,M803205692,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0,1,0,0,0,0,0,16


In [49]:
off_buycnt_normal_final.to_csv('off_buycnt_normal_final.csv',index=False)
# on_buycnt_normal_n_final.to_csv('on_buycnt_normal_n_final.csv',index=False)